# Полиномиальная регрессия (Polynomial Regression)

In [7]:
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
# возьмем данные о домах в Калифорнии
from sklearn.datasets import fetch_california_housing

In [8]:
data = fetch_california_housing(as_frame=True)

housing_data = data['frame']
housing_data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [9]:
# признаки
X = housing_data.drop(['MedHouseVal'], axis=1)
# целевой признак
y = housing_data['MedHouseVal']

# разделяем на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

> **Полиномиальная регрессия (Polynomial Regression)** — это более сложная модель, чем линейная регрессия. \
Вместо уравнения прямой используется уравнение полинома (многочлена). \
Степень полинома может быть сколь угодно большой: чем больше степень, тем сложнее модель.

Например, если в исходных данных было два признака $x_1$ и $x_2$.\
То после генерации полиномиальных признаков мы получим 5 признаков:\
$ x_1, x_2, x_1^2, x_2^2, x_1x_2,$

Полиномиальные признаки можно создать с помощью объекта класса [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) 
из модуля `preprocessing`. \
Это преобразователь, который позволяет сгенерировать полиномиальные признаки любой степени \
и добавить их в таблицу. \
У него есть два важных параметра:

- `degree` — степень полинома. По умолчанию используется степень `2` \
(лучше не указывать больше `3`-х, так как возрастает риск переобучения).
- `include_bias` — включать ли в результирующую таблицу столбец из единиц (x в степени 0). \
По умолчанию стоит `True`, лучше выставить его в значение `False`, \
так как столбец из единиц и так добавляется в методе наименьших квадратов.

In [10]:
# cоздаём генератор полиномиальных признаков
polynomial_features_generator = preprocessing.PolynomialFeatures(degree=2, include_bias=False)
# обучаем его
polynomial_features_generator.fit(X_train)

# генерируем полиномиальные признаки для тренировочной выборки
X_train_poly = polynomial_features_generator.transform(X_train)

# генерируем полиномиальные признаки для тестовой выборки
X_test_poly = polynomial_features_generator.transform(X_test)

# посмотрим на количество созданных признаков
print(X_train_poly.shape[1])
print(X_test_poly.shape[1])

44
44


Тип возвращаемых данных - `numpy`-массив.

In [11]:
print(type(X_train_poly))
print(type(X_test_poly))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


Передадим новые признаки в линейную регрессию.

In [12]:
# создаём объект класса LinearRegression
linear_regression_poly_model = linear_model.LinearRegression()

# обучаем модель
linear_regression_poly_model.fit(X_train_poly, y_train)

# делаем предсказание для тренировочной выборки
y_train_pred_poly = linear_regression_poly_model.predict(X_train_poly)

# делаем предсказание для тестовой выборки
y_test_pred_poly = linear_regression_poly_model.predict(X_test_poly)

# посчитаем MAPE для тренировочной выборки
mape_train = metrics.mean_absolute_percentage_error(y_train, y_train_pred_poly)
# посчитаем MAPE для тренировочной выборки
mape_test = metrics.mean_absolute_percentage_error(y_test, y_test_pred_poly)

print('MAPE train:', mape_train)
print('MAPE test:', mape_test)

MAPE train: 0.26165421173353975
MAPE test: 0.26992984572316


Метрики примерно одинаковы, значит переобучения нет.\
Значение метрики лучше, чем для [обычной линейной регрессии](./linear_regression.ipynb) (0.31978371646123604)